# Evidencia 2

Nombres: Diego Lira García
        Andrea Medina Rico

## Importaciones

In [ ]:
%pip install mesa==2.3.1 --quiet
%pip install matplotlib numpy pandas --quiet
%pip install seaborn --quiet

In [ ]:
# Importamos las clases que se requieren para manejar los agentes (Agent) y su entorno (Model).
# Cada modelo puede contener múltiples agentes.
from mesa import Agent, Model

# Debido a que necesitamos que existe un solo agente por celda, elegimos ''SingleGrid''.
from mesa.space import SingleGrid

# Con ''SimultaneousActivation, hacemos que todos los agentes se activen ''al mismo tiempo''.
from mesa.time import SimultaneousActivation

# Haremos uso de ''DataCollector'' para obtener información de cada paso de la simulación.
from mesa.datacollection import DataCollector

# Librería para modificar parámetros --> Corre varias simulaciones
from mesa.batchrunner import batch_run

# matplotlib lo usaremos crear una animación de cada uno de los pasos del modelo.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Importamos los siguientes paquetes para el mejor manejo de valores numéricos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

## 1. Solución aleatoria

In [ ]:
class FirefighterAgent(Agent):
    def __init__(self, id, model):
        super().__init__(id, model)

        self.action_points = 4
        self.carrying_victim = False
        self.knocked_down = False

    
    def step(self):
        while self.action_points > 0:
            # Asignar un número a cada función y que elija aleatoriamente
            option = self.random()
            if option == 0:
                self.move_to_cell()

            elif option == 1:
                self.toggle_door(self.pos)

            elif option == 2:
                self.extinguish(self.pos, 1)
            elif option == 3:
                self.extinguish(self.pos, 2)
            elif option == 4:
                self.extinguish(self.pos, 3)

            elif option == 5:
                self.chop(self.pos, self.random())

            elif option == 6:
                self.toggle_victim()
            else:
                self.save_points()


    def move_to_cell(self):
        """ Mueve al agente a una celda adyacente que no esté bloqueada por una puerta o pared. """
        possible_positions = self.model.grid.get_neighborhood(self.pos, moore = False, include_center = False)
        indexes = [x for x in range( len(possible_positions) )]
        np.random.shuffle(indexes)

        for i in indexes:
            position = possible_positions[i]
            status = self._get_cell_status(position)
            walls_doors = self._check_walls_doors(position, i)

            if not walls_doors:
                if status == 2 and not self.carrying_victim:
                    self.model.grid.move_agent(self, position)
                    self.action_points -= 2
                elif self.carrying_victim and status != 2:
                    self.action_points -= 2
                    self.model.grid.move_agent(self, position)
                else:
                    self.action_points -= 1
                    self.model.grid.move_agent(self, position)
                break


  
    def toggle_door(self,position):
        """ Cambia el estado de la puerta en la celda (x, y). """
        x, y = position
        cell = self.model.doors[x][y]

        # Obtener cuatro posible celdas  --> 0 Arriba, 1 derecha, 2 abajo, 3 izquierda
        possible_positions = self.model.grid.get_neighborhood(self.pos, moore = False, include_center = False)

        # Encontrar la puerta que se quiere alterar
        # 0 Arriba, 1 izquierda, 2 abajo, 3 derecha
        # 1 Abierto 2 Cerrado
        index = 0
        for i in cell:
            if i != 0:
                if i == 1:
                    cell[index] = 2
                    new_state = 2
                else:
                    cell[index] = 1
                    new_state = 1

                self.action_points -= 1
                break
            index += 1
        
        # Encontrar la otra celda de la puerta
        if index == 0:
            other_cell = possible_positions[0]
            other_index = 2
        elif index == 2:
            other_cell = possible_positions[2]
            other_index = 0
        elif index == 1:
            other_cell = possible_positions[3]
            other_index = 3
        elif index == 3:
            other_cell = possible_positions[1]
            other_index = 1
        else:
            return
        
        # Cambiar el estado de la otra celda
        next_cell = self.model.doors[other_cell[0]][other_cell[1]]
        next_cell[other_index] = new_state
        return
    
    
    def extinguish(self, position, option):
        """ Extingue el fuego o el humo en la celda (x, y). """
        x, y = position
        cell = self.model.grid_status[x][y]
        
        if option == 1 and cell == 2 and self.action_points >= 2:
            self.model.grid_status[x][y] = 0
            self.action_points -= 2
        elif option == 2 and cell == 2 and self.action_points >= 1:
            self.model.grid_status[x][y] = 1
            self.action_points -= 1
        elif option == 3 and cell == 1 and self.action_points >= 1:
            self.model.grid_status[x][y] = 0
            self.action_points -= 1
        else:
            return
            

    def chop(self, position, index):
        """ Daña la pared en la celda (x, y). """
        x, y = position
        xs, ys = self.pos
        wall = self.model.walls[x][y][self._get_contrapart(index)]
        
        if wall > 0:
            self.model.walls[x][y][self._get_contrapart(index)] -= 0.5
            self.action_points -= 2
            self.model.walls[xs][ys][index] -= 0.5


    def toggle_victim(self):
        """ Toma o deja a la víctima en la celda (x, y). """
        x, y = self.pos
        cell = self.model.poi[x][y]

        if cell == 2 and not self.carrying_victim:
            self.carrying_victim = True
            self.model.poi[x][y] = 0

        elif cell == 0 and self.carrying_victim:
            self.carrying_victim = False
            self.model.poi[x][y] = 2


    #def save_points(self):
        #if self.action_points <= 4:

        


    #Metodos auxiliares

    def _check_walls_doors(self, position, index):
        """ Verifica si la celda (x, y) tiene una pared o puerta que impida el paso. """
        x, y = position
        
        cell_doors = self.model.doors[x][y]
        cell_walls = self.model.walls[x][y]
        
        door = cell_doors[self._get_contrapart(index)]
        wall = cell_walls[self._get_contrapart(index)]
            
        if door == 2 or wall == 1 or wall == 0.5:
            return True
        else:
            return False
        
        
    def _get_cell_status(self, position):
        """ Obtiene el contenido de una celda (humo o fuego). """
        x, y = position
        return self.model.grid_status[x][y]
    
    
    def _get_contrapart(self, index):
        """ Obtiene qué indice de la celda es la contraparte del índice actual. """
        if index == 0:
            return 2
        elif index == 1:
            return 1
        elif index == 2:
            return 0
        elif index == 3:
            return 3
        

    def random(self):
        """ Regresa un número aleatorio entre 0 y 7. """
        return np.random.randint(0, 7)

In [ ]:
class PointOfInterest():
    def __init__(self, value):
        self.flipped = False
        self.value = value

In [ ]:
class FlashPointModel(Model):

    def __init__(self, walls_matrix, doors_matrix, poi_matrix, status_matrix, 
                 width = 10, height = 8, agents = 6, damage_counters = 24):

        super().__init__()
        self.grid = SingleGrid(width, height, torus = False)
        self.schedule = SimultaneousActivation(self)

        self.datacollector = DataCollector()

        # Matrices
        self.walls = walls_matrix
        self.doors = doors_matrix
        self.poi = poi_matrix
        self.grid_status = status_matrix

        self.saved_victims = 0
        self.lost_victims = 0
        self.damage_counters = damage_counters
        self.false_alarms = 6
        self.total_victims = 12
        self.point_of_interest = 3
        self.steps = 0

        self.edge_positions = self.get_edge_positions(width, height)

        # Creación agentes
        for i in range(agents):
            agent = FirefighterAgent(i, self)
            self.schedule.add(agent)
            # Posicionar
            self.grid.move_agent_to_one_of(agent, self.edge_positions, selection = 'random')


    def get_edge_positions(self, width, height):
        edge_positions = []
        for x in range(width):
            edge_positions.append((x, 0))  # Orilla superior
            edge_positions.append((x, height - 1))  # Orilla inferior

        for y in range(1, height - 1):
            edge_positions.append((0, y))  # Orilla izquierda
            edge_positions.append((width - 1, y))  # Orilla derecha

        return edge_positions
    
    def advance_fire(self):
        """ Coloca un humo aletorio. """
        x, y = self.roll_dice()
        cell = self.grid_status[x][y]
        
        # Si la celda está vacía, se coloca humo
        if cell == 0:
            self.grid_status[x][y] = 1
        elif cell == 1:
            self.grid_status[x][y] = 2
        elif cell == 2:
            self.explosion(x, y)
            
    def explosion(self, x, y):
        """ Realiza una explosión y llama a explosion_effect para las celdas adyacentes. """
        possible_positions = self.model.grid.get_neighborhood(self.pos, moore = False, include_center = False)
        
        index = 0
        for position in possible_positions:
            self.explosion_effect(position[0], position[1], index, x, y)
            index += 1
        
        
    def explosion_effect(self, x, y, index, current_x, current_y):
        """ Verifica si hay una celda adyacente que tenga fuego. """
        cell = self.grid_status[x][y]
        other_index = self._get_contrapart(index)
        door = self.doors[x][y][other_index]
        wall = self.walls[x][y][other_index]
        
        if door == 2:
            self.doors[x][y][other_index] = 0
            self.doors[current_x][current_y][index] = 0  
        elif door == 1:
            self.doors[x][y][other_index] = 0
            self.doors[current_x][current_y][index] = 0
            self.grid_status[x][y] = 2
        elif wall == 1 or wall == 0.5:
            self.walls[x][y][other_index] -= 0.5
            self.walls[current_x][current_y][index] -= 0.5
        elif wall == 0:
            if cell == 2:
                self.shockwave(x, y)
            else:
                self.grid_status[x][y] = 2
                
                
    def shockwave(self, x, y):
        """ Realiza una onda expansiva de llammas."""
        
        

        
        
    def _get_contrapart(self, index):
        """ Obtiene qué indice de la celda es la contraparte del índice actual. """
        if index == 0:
            return 2
        elif index == 1:
            return 1
        elif index == 2:
            return 0
        elif index == 3:
            return 3
        
        
    def roll_dice(self):
        """ Regresa un número aleatorio entre 1 y 8 y del 1 al 6. """
        return self.random.randint(1, 8), self.random.randint(1, 6)
